# Análisis de variable precio

## Importación de librerías y datos

Por medio de nuestra libería ESIOS_contoller.py importamos nuestro último dataset de datos y lo parseamos para su uso. Sirve tanto como para Drive como jupiter.

In [56]:
import json, urllib, datetime, pickle, time
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import *
from keras.models import *
from keras.layers import *
from sklearn.preprocessing import *
from keras.optimizers import *
from scipy.stats import *
from importlib.machinery import SourceFileLoader


try:
  from google.colab import drive
  drive.mount('/content/drive')
  path = '/content/drive/My Drive/TFM/Utils/ESIOS_contoller.py'
  in_colab = True
except:
  path = '../utils/ESIOS_contoller.py'
  in_colab = False
  

esios_assembler = SourceFileLoader('esios', path).load_module()

esios_controller = esios_assembler.ESIOS(in_colab)
data_consumo = esios_controller.get_data()
data_consumo.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mostrando los datos de data_total.csv
(30555, 29)
________________________________________________________________________________


,fecha,PVPC_DEF,PVPC_2_PED_NOC,PVPC_ELEC_NOC,date_timestamp,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,date_day,Brent_price,Holiday,Precio de Regulación Secundaria subir,Precio de Regulación Secundaria bajar,Precio mercado SPOT Diario_x,Demanda real,Generación prevista Solar,Saldo total interconexiones programa p48,Generación programada P48 Exportación Portugal,Generación programada P48 Exportación Francia,Generación programada P48 Importación Portugal,Generación programada P48 Importación Francia,Precio SPOT PT,Precio SPOT FR
0,2015-10-01 00:00:00+02:00,117.77,69.49,71.88,1.443650e+09,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,2015-10-01,47.48,0.0,61.18,49.13,56.65,25305.0,267.6,182.4,-606.6,-351.4,1232.1,1200.0,56.65,38.56
1,2015-10-01 01:00:00+02:00,117.09,68.21,65.57,1.443654e+09,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,2015-10-01,47.48,0.0,59.59,49.13,53.50,23959.0,195.7,-269.5,-1112.5,-351.4,1232.1,1200.0,53.50,31.27
2,2015-10-01 02:00:00+02:00,114.59,65.34,62.53,1.443658e+09,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,2015-10-01,47.48,0.0,56.27,49.13,49.69,22873.0,202.1,94.0,-951.0,-351.4,1232.1,1200.0,49.69,30.66
3,2015-10-01 03:00:00+02:00,116.60,66.53,63.40,1.443661e+09,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,2015-10-01,47.48,0.0,45.99,45.50,49.10,22550.0,61.3,319.7,-846.3,-351.4,1232.1,1200.0,49.10,29.14
4,2015-10-01 04:00:00+02:00,122.07,71.03,67.37,1.443665e+09,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,2015-10-01,47.48,0.0,52.28,49.13,51.25,22017.0,45.1,405.5,-760.5,-351.4,1232.1,1200.0,51.25,29.40


## Selección de variables

### Selección de variables en tiempo real.

In [49]:
data_x = esios_controller.get_data_real_time()

data_y = data_consumo[['PVPC_DEF']]
data_x.head()


,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,Brent_price,Holiday,Precio de Regulación Secundaria subir,Precio de Regulación Secundaria bajar,Generación prevista Solar,Saldo total interconexiones programa p48,Generación programada P48 Exportación Portugal,Generación programada P48 Exportación Francia,Generación programada P48 Importación Portugal,Generación programada P48 Importación Francia
0,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,47.48,0.0,61.18,49.13,267.6,182.4,-606.6,-351.4,1232.1,1200.0
1,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,47.48,0.0,59.59,49.13,195.7,-269.5,-1112.5,-351.4,1232.1,1200.0
2,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,47.48,0.0,56.27,49.13,202.1,94.0,-951.0,-351.4,1232.1,1200.0
3,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,47.48,0.0,45.99,45.50,61.3,319.7,-846.3,-351.4,1232.1,1200.0
4,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,47.48,0.0,52.28,49.13,45.1,405.5,-760.5,-351.4,1232.1,1200.0


In [50]:
data_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30555 entries, 0 to 30554
Data columns (total 19 columns):
Demanda                                           30555 non-null float64
Eolica                                            30555 non-null float64
Nuclear                                           30555 non-null float64
Solar                                             30555 non-null float64
Solar_Fotovoltaica                                30555 non-null float64
Solar_Termica                                     30555 non-null float64
Termica_Renovable                                 30555 non-null float64
Prevista                                          30555 non-null float64
Programada                                        30555 non-null float64
Brent_price                                       30555 non-null float64
Holiday                                           30555 non-null float64
Precio de Regulación Secundaria subir             30555 non-null float64
Precio de Regul

### Eliminar variables con baja varianza

In [51]:
from sklearn.feature_selection import VarianceThreshold

var_th = VarianceThreshold(threshold = 0.60)
x_var = var_th.fit_transform(data_x)

print('Variables originales ', data_x.shape[1])
print('Variables finales ', x_var.shape[1])

print('Listado de variables ', np.asarray(list(data_x))[var_th.get_support()])

Variables originales  19
Variables finales  18
Listado de variables  ['Demanda' 'Eolica' 'Nuclear' 'Solar' 'Solar_Fotovoltaica' 'Solar_Termica'
 'Termica_Renovable' 'Prevista' 'Programada' 'Brent_price'
 'Precio de Regulación Secundaria subir'
 'Precio de Regulación Secundaria bajar' 'Generación prevista Solar'
 'Saldo total interconexiones programa p48'
 'Generación programada P48 Exportación Portugal'
 'Generación programada P48 Exportación Francia'
 'Generación programada P48 Importación Portugal'
 'Generación programada P48 Importación Francia']


## Selección usando Usando un numero K

In [54]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import chi2

var_sk = SelectKBest(f_regression, k = 10)
x_sk = var_sk.fit_transform(data_x, data_y)
 
print("Variables finales ", x_sk.shape[1])
 
print("Listado de variables ", np.asarray(list(data_x))[var_sk.get_support()])

Variables finales  10
Listado de variables  ['Demanda' 'Eolica' 'Termica_Renovable' 'Prevista' 'Programada'
 'Brent_price' 'Holiday' 'Precio de Regulación Secundaria subir'
 'Precio de Regulación Secundaria bajar'
 'Saldo total interconexiones programa p48']


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Selección de mejores k 

In [55]:
var_sp = SelectPercentile(f_regression, percentile = 75)
x_sp = var_sp.fit_transform(data_x, data_y)
 
print("Variables finales ", x_sp.shape[1])
 
print("Listado de variables ", np.asarray(list(data_x))[var_sp.get_support()])

Variables finales  14
Listado de variables  ['Demanda' 'Eolica' 'Solar' 'Solar_Termica' 'Termica_Renovable' 'Prevista'
 'Programada' 'Brent_price' 'Holiday'
 'Precio de Regulación Secundaria subir'
 'Precio de Regulación Secundaria bajar' 'Generación prevista Solar'
 'Saldo total interconexiones programa p48'
 'Generación programada P48 Importación Francia']


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
